# Examples of Integration With Fabric Onelake
**Note**: This notebook can be run with a azure machine learning compute instance

In [ ]:
!pip install mltable

## Code Sample to query lakehouse table

Compute Instance needs contributor rights to the fabric workspace - View permissions is not enough (doesn't spark access)

In [7]:
from mltable import from_delta_lake
path="abfss://UnifiedData@onelake.dfs.fabric.microsoft.com/maag_bronze.Lakehouse/Tables/dbo/ml-creditcard-as-default-payment-prediction"  # path copied directly from the table properties in lakehouse

df1 = from_delta_lake(path).to_pandas_dataframe()

df1.head()  # Shows the first 5 rows

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default_payment_next_month
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


## Code Sample to query lakehouse table from a specific point in time

Compute Instance needs contributor rights to the fabric workspace - View permissions is not enough (doesn't spark access)

In [8]:
from mltable import from_delta_lake
path="abfss://UnifiedData@onelake.dfs.fabric.microsoft.com/maag_bronze.Lakehouse/Tables/dbo/ml-creditcard-as-default-payment-prediction"  # path copied directly from the table properties in lakehouse

df1 = from_delta_lake(path, timestamp_as_of="2022-10-01T00:00:00Z").to_pandas_dataframe()

df1.head()  # Shows the first 5 rows

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default_payment_next_month
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


### Plug-n-Play Onelake storage as alternative to Blob storage/ADSL Gen 2 Code

 #### Sample of how to register a Fabric Lakehouse (Onelake) as a Data store 
 The workspace and lakehouse guids can be extracted from the URL when you browse a lakehouse in your browser - the first guid is the lakehouse, the second is your lakehouse - see the documentation for more info

In [9]:
#Create a data store - Can also be done from UI via Connections

from azure.ai.ml.entities import OneLakeDatastore, OneLakeArtifact
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient.from_config(credential=DefaultAzureCredential())

store = OneLakeDatastore(
    name="onelake_example_id", #the name of the data store we are going to make
    description="Datastore pointing to an Microsoft fabric artifact.",
    one_lake_workspace_name="ba00abf2-6be0-4676-928b-b89bb242e1bf", #{your_one_lake_workspace_guid}
    endpoint="onelake.dfs.fabric.microsoft.com", #{your_one_lake_endpoint}
    artifact = OneLakeArtifact(
        name="9d090bd3-f727-4028-a4f7-c026027d5597/Files", #{your_one_lake_artifact_guid}/Files
        type="lake_house"
    )
)

ml_client.create_or_update(store)

Found the config file in: /config.json


OneLakeDatastore({'type': <DatastoreType.ONE_LAKE: 'OneLake'>, 'name': 'onelake_example_id', 'description': 'Datastore pointing to an Microsoft fabric artifact.', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/<subscription-id>/resourceGroups/rg-we-atpws-aml/providers/Microsoft.MachineLearningServices/workspaces/aml-ws-atp001/datastores/onelake_example_id', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/atpcomputeinstance001/code/Users/jeffreyadmin', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7054e6434880>, 'credentials': <azure.ai.ml.entities._credentials.NoneCredentialConfiguration object at 0x7054e64340a0>, 'artifact': {'name': '9d090bd3-f727-4028-a4f7-c026027d5597/Files', 'type': 'lake_house'}, 'one_lake_workspace_name': 'ba00abf2-6be0-4676-928b-b89bb242e1bf', 'endpoint': 'onelake.dfs.fabric.microsoft.com'})

 #### Sample code of how to access files in a data store (generic for all AML solutions)

In [6]:
#Generated by the AML UI:
# Data --> Data Stores --> Choose the registered Onelake Data Store --> Browse to the file you want to use --> Copy usage code

import pandas as pd
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient.from_config(credential=DefaultAzureCredential())

#URI = "azureml://subscriptions/<Subscripion ID>/resourcegroups/<Resource group Name>/workspaces/<azure machine learning workspace resource name>/datastores/<Your Onelake Datstore name>/paths/<a subfolder in your Files/ folder in your lakehouse>/<your file.csv>"
uri = "azureml://subscriptions/<subscription-id>/resourcegroups/rg-we-atpws-aml/workspaces/aml-ws-atp001/datastores/onelake_example_id/paths/raw-files/ml-creditcard-as-default-payment-prediction/default_of_credit_card_clients.csv"

df = pd.read_csv(uri)
df

Found the config file in: /config.json
Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/mlflow/__init__.py:41: UserWarning: Versions of mlflow (3.1.1) and mlflow-skinny (2.22.1) are different. This may lead to unexpected behavior. Please install the same version of both packages.
  mlflow.mismatch._check_version_mismatch()
Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already

Resolving access token for scope "https://storage.azure.com/.default" using identity of type "MANAGED".
Getting data access token with Assigned Identity (client_id=clientid) and endpoint type based on configuration
Resolving access token for scope "https://storage.azure.com/.default" using identity of type "MANAGED".
Getting data access token with Assigned Identity (client_id=clientid) and endpoint type based on configuration


,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default_payment_next_month
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,29996,220000,1,3,1,39,0,0,0,0,...,88004,31237,15980,8500,20000,5003,3047,5000,1000,0
29996,29997,150000,1,3,2,43,-1,-1,-1,-1,...,8979,5190,0,1837,3526,8998,129,0,0,0
29997,29998,30000,1,2,2,37,4,3,2,-1,...,20878,20582,19357,0,0,22000,4200,2000,3100,1
29998,29999,80000,1,3,1,41,1,-1,0,0,...,52774,11855,48944,85900,3409,1178,1926,52964,1804,1


# API access to a delta table through GraphQL endpoint

In [ ]:
from azure.identity import ClientSecretCredential
import requests
import json

# Service Principal credentials (spn)

# spn has been given following api permission in Azure:
#Lakehouse.ReadWrite.All - Delegated
#GraphQL.Execute.All - Delegated
#GraphQLApi.Execute.All - Delegated
#GraphQLApi.ReadWrite.All - Delegated

tenant_id = "<tenant-id>"
client_id = "<client-id>"
client_secret = "<client-secret>"

# Correct scope for Fabric API
scope = "https://api.fabric.microsoft.com/.default"

# Authenticate using ClientSecretCredential
credential = ClientSecretCredential(tenant_id, client_id, client_secret)

# Get token
token = credential.get_token(scope)

if not token.token:
    raise Exception("Could not get access token")

# Prepare headers
headers = {
    "Authorization": f"Bearer {token.token}",
    "Content-Type": "application/json"
}

# Fabric GraphQL 
# endpoint was gathered through the "Geneate Code" button in the UI of a created GraphQL item in Fabric 
endpoint = "https://ba00abf26be04676928bb89bb242e1bf.zba.graphql.fabric.microsoft.com/v1/workspaces/ba00abf2-6be0-4676-928b-b89bb242e1bf/graphqlapis/88a73776-cbf0-43bd-a9db-47a3e330488d/graphql"

# GraphQL query example
#the following queries a lakehouse table know as mlcreditcardasdefaultpaymentpredictions - retrieve only top 10, filtered by sex=1 and age>30 for a subset of columns
query = """
query {
  mlcreditcardasdefaultpaymentpredictions(
    first: 10
    filter: { SEX: { eq: 1 }, AGE: { gt: 30 } }
  ) {
    items {
      ID
      LIMIT_BAL
      SEX
      EDUCATION
      MARRIAGE
      AGE
    }
  }
}
"""

# Issue request
response = requests.post(endpoint, json={"query": query}, headers=headers)
response.raise_for_status()
print(response.json())

{'data': {'mlcreditcardasdefaultpaymentpredictions': {'items': [{'ID': 5, 'LIMIT_BAL': 50000, 'SEX': 1, 'EDUCATION': 2, 'MARRIAGE': 1, 'AGE': 57}, {'ID': 6, 'LIMIT_BAL': 50000, 'SEX': 1, 'EDUCATION': 1, 'MARRIAGE': 2, 'AGE': 37}, {'ID': 10, 'LIMIT_BAL': 20000, 'SEX': 1, 'EDUCATION': 3, 'MARRIAGE': 2, 'AGE': 35}, {'ID': 18, 'LIMIT_BAL': 320000, 'SEX': 1, 'EDUCATION': 1, 'MARRIAGE': 1, 'AGE': 49}, {'ID': 32, 'LIMIT_BAL': 50000, 'SEX': 1, 'EDUCATION': 2, 'MARRIAGE': 2, 'AGE': 33}, {'ID': 33, 'LIMIT_BAL': 100000, 'SEX': 1, 'EDUCATION': 1, 'MARRIAGE': 2, 'AGE': 32}, {'ID': 35, 'LIMIT_BAL': 500000, 'SEX': 1, 'EDUCATION': 1, 'MARRIAGE': 1, 'AGE': 58}, {'ID': 37, 'LIMIT_BAL': 280000, 'SEX': 1, 'EDUCATION': 2, 'MARRIAGE': 1, 'AGE': 40}, {'ID': 40, 'LIMIT_BAL': 280000, 'SEX': 1, 'EDUCATION': 1, 'MARRIAGE': 2, 'AGE': 31}, {'ID': 41, 'LIMIT_BAL': 360000, 'SEX': 1, 'EDUCATION': 1, 'MARRIAGE': 2, 'AGE': 33}]}}}
